In [1]:
import os
import json
import glob
import shutil
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.font_manager import FontProperties

In [2]:
degrees = ['15cm_0mm_0deg', '15cm_25mm_5deg', '15cm_50mm_10deg', '20cm_0mm_0deg', '20cm_25mm_5deg', '20cm_50mm_10deg']
acupoints_name = ["HX6(肛門)", "HX3(尿道)", "HX1(耳中)", "TF4(神門)", "AH9(腰柢椎)", "AH11(胸椎)", "CO5(十二指腸)", 
                  "CO4(胃)", "CO3(賁門)", "CO14(肺)", "CO15(心)", "CO18(內分泌)", "HX2(輪2)", "AT2(緣中)", 
                  "LO8(扁桃體)", "LO1(牙)" , "LO2(舌)", "LO3(頷)", "LO4(垂前)", "LO5(眼)", "LO6(內耳)"]
ear_types = ["free","attached"]
cm = {"cather":4.72, "daniel":4.74, "dominic":3.9, "jack":3.91, "jakaria":4.20, "jimmy":4.60, "vicky":4.69, "wayne":4.75,
      "gaby":4.05, "matt":5.0, "toby":4.26, "wendy":4.14, "willy":4.51}

In [3]:
class CDF():
    def __init__(self, src):
        self.src = src
    def read_error_csv(self, fpath1):
        error_np = pd.read_csv(fpath1, index_col = 0).to_numpy()
        return error_np
    def cal_CDF(self, src, ear_types, acupoints_name, degrees):
        final = {}
        for ear_type in ear_types:
            final[ear_type] ={}
            total = []
            names = os.listdir(os.path.join("..", src, ear_type, "result"))
            for name in names:
                for deg in degrees:
                    error_file = glob.glob(os.path.join("..", src, ear_type, "result", name, "error", deg, "*.csv"))[0]
                    error_np = self.read_error_csv(error_file)

                    interval = len(error_np) / 3
                    index = [0, int(interval), int(interval*2), len(error_np)-1]
                    for i in index:
                        total.append(error_np[i])
            total = np.array(total)
            total = total.reshape(-1)
            sorted_total = np.sort(total*10)
            cumulative_prob_total = np.arange(1, len(total) + 1) / len(total)
            final[ear_type]["sorted_error"] = sorted_total
            final[ear_type]["cumulative_prob"] = cumulative_prob_total
        return final     

In [4]:
class CDF_Draw():
    def __init__(self, CDF_result_0, CDF_result_1):
        self.CDF_result_0 = CDF_result_0
        self.CDF_result_1 = CDF_result_1
        
    def Draw_one_CDF(self, CDF_result, title, label, ear_types=["free","attached"]):
        for ear_type in ear_types:
            plt.plot(CDF_result[ear_type]["sorted_error"], CDF_result[ear_type]["cumulative_prob"], label="The result of {label_}".format(label_ = label))
            plt.xlim(0, 10)
            plt.ylim(0, 1.1)
            plt.xlabel("mm")
            plt.ylabel("Probability")
            plt.title(title)
            plt.legend()
            plt.show()
#             if not os.path.isdir(os.path.join("../mmpose/k_fold", "CDF", ear_type)):
#                 os.makedirs(os.path.join("../mmpose/k_fold", "CDF", ear_type))
#             plt.savefig(os.path.join("../mmpose/k_fold", "CDF", ear_type, 'CDF.png').format(index = 0))
            
    def Draw_two_CDF(self, CDF_result_0, CDF_result_1, title, label_0, label_1, ear_types=["free","attached"]):
        for ear_type in ear_types:
            plt.plot(CDF_result_0[ear_type]["sorted_error"], CDF_result_0[ear_type]["cumulative_prob"], label="The result of {label_}".format(label_ = label_0))
            plt.plot(CDF_result_1[ear_type]["sorted_error"], CDF_result_1[ear_type]["cumulative_prob"], label="The result of {label_}".format(label_ = label_1))
            plt.xlim(0, 10)
            plt.ylim(0, 1.1)
            plt.xlabel("mm")
            plt.ylabel("Probability")
            plt.title(title)
            plt.legend()
            plt.show()
#             if not os.path.isdir(os.path.join("../mmpose/k_fold", "CDF", ear_type)):
#                 os.makedirs(os.path.join("../mmpose/k_fold", "CDF", ear_type))
#             plt.savefig(os.path.join("../mmpose/k_fold", "CDF", ear_type, 'CDF.png').format(index = 0))